<a href="https://colab.research.google.com/github/AbadeC/ProjetoImersaoAI/blob/main/ProjetoImersaoAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introdução



A proposta deste projeto é criar um Chatbot que responda as perguntas dos usuários com base em PDFs preparados como Base de Conhecimento. Seus usos serão em interação com Clientes, Fornecedores, comunidade ou mesmo para ajudar estudantes em seus estudos.

### Funcionamento
O funcionamento do programa é simples:
- Preparando o Ambiente: realizadas as instalações e importações, configurados os parâmetros;
- Preparando a Base de Conhecimento: São realizados os uploads de arquivo para a área de trabalho do colab (/content/). Podem ser enviados um ou mais arquivos em PDF, preparados especificamente para servir como base de conhecimento ou não (a eficiência da ferramenta depende disso);
- Executando o trabalho: é aqui que acontece a interação entre o usuário e o chat. Para terminar, teclar ENTER duas vezes consecutivas;
- Auxiliares e uso futuro: rotinas para lidar com os arquivos carregados (mostrar e apagar) e para serem usadas futuramente.

### Melhorias futuras
As possibilidades de melhoria são muitas, commo por exemplo:
* Ampliação da Base de Conhecimento opcionais:
 - Links externos
 - Outros tipos de arquivo
 - Imagens
* Interface amigável
* Inclusão de ícone de avatar para o Chatbot
* Parametrizações de acordo com o interesse do Usuário:
 - Personalidade
 - Estilo
 - Escolha de um nome para o Chatbot


# 2. Preparando o Ambiente

In [225]:
#Instalando bibliotecas externas
!pip install -q -U google-generativeai #SDK do Gemini
!pip install PyPDF2     #Para os PDFs

In [226]:
#Importações de bibliotecas já embutidas no Colab e configurações iniciais
import textwrap
from IPython.display import display
from IPython.display import Markdown
from pathlib import Path
import hashlib

from IPython.display import Image   # Para a imagem do Zig

# Importação do SDK do Python
import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [227]:
# Inicializando o Modelo (1)
# Definições básicas para o Modelo

generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

In [228]:
# Variáveis úteis

# Instruções de sistema: Definindo as características e a personalidade do Chatbot.
system_instruction = "Você é um assistente de bate-papo prestativo chamado ZIG. Seja gentil, educado e entusiasta. Responda sempre as perguntas em português brasileiro. Aja como um especialista nos assuntos das Bases de Conhecimento a que tem acesso e utilize o contexto fornecido para responder às perguntas do usuário da forma mais completa e precisa possível. Apresente as informações de forma clara, concisa e organizada. Se a resposta puder ser encontrada em diferentes partes do contexto, combine as informações relevantes. Se a pergunta se referir a requisitos ou documentos, assegure-se de incluir tudo, sem omitir nada. Se a resposta não estiver no contexto, diga que não a sabe, peça desculpas e sugira que o usuário entre em contato com o suporte para obter informações. Lembre-se: Responda apenas com base no contexto fornecido. Mantenha um tom positivo e encorajador ao interagir com o usuário."

url = "https://i.postimg.cc/nz4ZskZn/Gemini-Generated-Image-ddohr6ddohr6ddoh-Edited-1.jpg"    # URL da imagem do Zig


In [24]:
# Inicializando o Modelo (2)

# Verificação inicial das disponibilidades
#for modelo in genai.list_models():
#  if 'generateContent' in modelo.supported_generation_methods:
#    print(modelo.name)

# Definição do Modelo a ser usado

# Opção 1
#model = genai.GenerativeModel(model_name="gemini-1.0-pro",
#                              generation_config=generation_config,
#                              system_instruction=system_instruction, #Instruções de personalidade
#                              safety_settings=safety_settings)

# Opção 2
#model = genai.GenerativeModel(model_name="1.0-pro-latest",
#                              generation_config=generation_config,
#                              system_instruction=system_instruction, #Instruções de personalidade
#                              safety_settings=safety_settings)

# Opção 3 - É a única que aceita as instruções iniciais personalizadas.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction, #Instruções de personalidade
                              safety_settings=safety_settings)

# 3. Preparando a Base de Conhecimento

In [230]:
# Alimentando a base de conhecimento com PDFs
from google.colab import files
import os

print('Image(url=url)')

Mensagem_carrega_PDF = 'Faça o **carregamento** (*upload*) de um ou mais arquivos em **PDF** (SELECIONE TODOS de uma só vez) que farão parte da Base de Conhecimento do Chatbot:'

# Verifica se já existem arquivos PDF no diretório de destino
arquivos_pdf_existentes = [arq for arq in os.listdir(diretorio_destino) if arq.endswith('.pdf')]

if arquivos_pdf_existentes:
  display(Markdown("Os seguintes arquivos PDF já estão no diretório:"))
  for arquivo in arquivos_pdf_existentes:
    print(f"- {arquivo}")

while True:
  display(Markdown(Mensagem_carrega_PDF))
  uploaded = files.upload()
  for filename in uploaded.keys():
    print(f'Arquivo "{filename}" carregado com sucesso.')
  arquivos_pdf_existentes = [arq for arq in os.listdir(diretorio_destino) if arq.endswith('.pdf')]
  if arquivos_pdf_existentes:
    display(Markdown("Base de Conhecimento preparada."))
    display(Markdown("Os seguintes arquivos PDF já estão no diretório:"))
    for arquivo in arquivos_pdf_existentes:
      print(f"- {arquivo}")
    break
  else:
    display(Markdown("**ALERTA:** Nenhum arquivo foi carregado ainda. Isso impede que eu funcione adequadamente."))
    continue

# Versão futura
# Alimentando a base de conhecimentos com links externos
#
# Alimentando a base de conhecimento com outros tipos de arquivo...
#

Image(url=url)


Faça o **carregamento** (*upload*) de um ou mais arquivos em **PDF** (SELECIONE TODOS de uma só vez) que farão parte da Base de Conhecimento do Chatbot:

**ALERTA:** Nenhum arquivo foi carregado ainda. Isso impede que eu funcione adequadamente.

Faça o **carregamento** (*upload*) de um ou mais arquivos em **PDF** (SELECIONE TODOS de uma só vez) que farão parte da Base de Conhecimento do Chatbot:

Saving FAQ de Energia.pdf to FAQ de Energia.pdf
Arquivo "FAQ de Energia.pdf" carregado com sucesso.


Base de Conhecimento preparada.

Os seguintes arquivos PDF já estão no diretório:

- FAQ de Energia.pdf


# 4. Executando o Trabalho

In [231]:
Image(url=url)

In [236]:
from pathlib import Path
import hashlib
import PyPDF2
import glob # Importe o módulo glob

Image(url=url)

def extract_pdf_pages(directory: str) -> list[str]:
    parts = []
    for filename in glob.glob(f"{directory}/*.pdf"):  # Encontre todos os PDFs no diretório
        parts.append(f"--- START OF PDF {filename} ---")

        # Lógica para ler o PDF e retornar uma lista de páginas:
        with open(filename, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            pages = []
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num].extract_text()
                pages.append(page)

        for index, page in enumerate(pages):
            parts.append(f"--- PAGE {index} ---")
            parts.append(page)
    return parts

convo = model.start_chat(history=[
    {
        "role": "user",
        "parts": extract_pdf_pages("/content/") # Passa o diretório
    }
])

# Início da interação

display(Markdown('Olá, sou o Zig. Estou aqui pra responder suas perguntas sobre a base de conhecimento carregada e sobre mim. 😊'))
display(Markdown('Se quiser interromper o chat, **tecle ENTER duas vezes seguidas** com o campo de escrita vazio. Como posso te ajudar?'))

entradas_vazias_consecutivas = 0    # Contador de tecla ENTER vazia

# Mantendo o diálogo

while entradas_vazias_consecutivas < 2:
  prompt = input('Escreva aqui: ')
  if prompt.strip() == "":
    entradas_vazias_consecutivas += 1
  else:
    entradas_vazias_consecutivas = 0
    convo.send_message(prompt)
    display(Markdown(convo.last.text))

# Encerramento

display(Markdown('Obrigado por ter interagido comigo. Volte sempre.'))

# Fim

Olá, sou o Zig. Estou aqui pra responder suas perguntas sobre a base de conhecimento carregada e sobre mim. 😊

Se quiser interromper o chat, **tecle ENTER duas vezes seguidas** com o campo de escrita vazio. Como posso te ajudar?

Escreva aqui: Qual o seu nome


Olá! Meu nome é ZIG, e estou aqui para te ajudar 😊. Como posso ser útil hoje? 😄 


Escreva aqui: Tchau


Tchau! Foi um prazer conversar com você. 😊  Espero que tenha um ótimo dia! 😄  Se precisar de algo mais, é só chamar! 😉 


Escreva aqui: 
Escreva aqui: 


Obrigado por ter interagido comigo. Volte sempre.

# 5. Auxiliares e uso futuro

Rotinas auxiliares para atividades de manutenção e fora do processo normal. Futuramente poderão ser incorporadas.

In [217]:
# Apagar os arquivos carregados anteriormente #####################################################

import os

# Lista os arquivos no diretório atual
arquivos_no_diretorio = os.listdir()

# Itera sobre os arquivos e apaga os arquivos PDF
for arquivo in arquivos_no_diretorio:
  if arquivo.endswith(".pdf"):  # Verifica se o arquivo é um PDF
    os.remove(arquivo)
    print(f'Arquivo "{arquivo}" apagado com sucesso.')

print("Não há ou todos os arquivos PDF foram apagados.")

Arquivo "FAQ de Energia.pdf" apagado com sucesso.
Arquivo "Orientações Gerais Grupo BNI BH Premium.pdf" apagado com sucesso.
Não há ou todos os arquivos PDF foram apagados.


In [218]:
# Mostra os arquivos na área de trabalho #######################################

import os

# Diretório de destino dos arquivos PDF
#diretorio_destino = '/content/'

# Lista os arquivos no diretório atual
arquivos_no_diretorio = os.listdir()

# Itera sobre os arquivos e exibe o caminho completo
for arquivo in arquivos_no_diretorio:
  caminho_completo = os.path.abspath(arquivo)  # Obtém o caminho completo do arquivo
  print(f'Arquivo: {arquivo}, Caminho: {caminho_completo}')

Arquivo: .config, Caminho: /content/.config
Arquivo: sample_data, Caminho: /content/sample_data


In [219]:
# Verifica se algum arquivo foi carregado ################################################ COM FALHA
if uploaded:
  display(Markdown("Os seguintes arquivos foram carregados:"))   #print("Os seguintes arquivos foram carregados:")
  for filename in uploaded.keys():
    print(f"- {filename}")
else:
  display(Markdown("**ALERTA:** Nenhum arquivo foi carregado ainda. Isso impede que eu funcione adequadamente."))   #print("Nenhum arquivo foi carregado ainda.")

Os seguintes arquivos foram carregados:

- Orientações Gerais Grupo BNI BH Premium.pdf


In [222]:
# Resposta mais descontraída. Uso futuro.
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {convo.last.text}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)

display(Markdown(response.text))

AttributeError: 'NoneType' object has no attribute 'text'